In [1]:
import os
import ee
import geemap
from pathlib import Path
from datetime import datetime, timedelta , date
import time
import pandas as pd
import csv
import glob
import folium
from IPython.display import display, clear_output
import shutil
import shapefile
import ipywidgets as widgets
from tkinter import Tk, filedialog
import traitlets
from ipyfilechooser import FileChooser

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True,filetypes=[('shp','.shp')])

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [3]:
# After executing this line of code for the first use, you can get the authentication number linked to Google.
Map = geemap.Map()
# Authenticate the Google earth engine with google account
ee.Initialize() 

In [4]:
# give the shp file
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [5]:
shp = shapefile.Reader("".join(my_button.files))
b = []
for i in range(len(shp.fields)):
    a = shp.fields[i][0]
    b.append(a)

# give the shapefile name

file_name = widgets.Dropdown(
options=b,
description='Regional category')

file_name

Dropdown(description='Regional category', options=('DeletionFlag', 'STATE'), value='DeletionFlag')

In [6]:
# give the star date and end date

star = widgets.DatePicker(
    description='Pick a Star Date',
    disabled=False
)
end = widgets.DatePicker(
    description='Pick a End Date',
    disabled=False
)

widgets.HBox([star, end])

In [7]:
# give the bands
#
band_name = widgets.SelectMultiple(
options=['ndvi','evi','savi','ndwi1','ndwi2','ndwi3', 'K', 'C'],
description='Band',
)

band_name

SelectMultiple(description='Band', options=('ndvi', 'evi', 'savi', 'ndwi1', 'ndwi2', 'ndwi3', 'K', 'C'), value…

In [8]:
statics =widgets.Dropdown(
    options=['MEAN','MAXIMUM', 'MINIMUM', 'MEDIAN', 'STD', 'VARIANCE', 'SUM'],
    value='MEAN',
    description='Statistics')

statics

Dropdown(description='Statistics', options=('MEAN', 'MAXIMUM', 'MINIMUM', 'MEDIAN', 'STD', 'VARIANCE', 'SUM'),…

In [9]:
# give the output floder and flie name
folder = FileChooser()
display(folder)


FileChooser(path='C:\Users\USER\Desktop\satellite python code\satellite python code\zonal statistic', filename…

In [25]:
# create folder
folder_name = 'data_all'

# create folder name : data_all
if os.path.isdir(folder_name) == True:
    shutil.rmtree(folder_name, ignore_errors=True)
    os.makedirs(folder_name)
else:
    os.makedirs(folder_name)


In [22]:
states = geemap.shp_to_ee("".join(my_button.files))

#Functionalize each parameter

def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.expression(
      '((NIR - RED) / (NIR + RED ))', {
      'NIR' : image.select('SR_B5').multiply(0.0001),
      'RED' : image.select('SR_B4').multiply(0.0001),}).rename("ndvi")
    
    return image.addBands(ndvi);

# get indexes NDWI1
def getNDWI1(image):
    
    ndwi1 = image.expression(
      '((NIR - SWIR1) / (NIR + SWIR1 ))', {
      'NIR' : image.select('SR_B5').multiply(0.0001),
      'SWIR1' : image.select('SR_B6').multiply(0.0001),}).rename("ndwi1")
    # Normalized difference vegetation index (NDWI1) (Xnir - Xswir1)/(Xnir + Xswir1)
    

    return image.addBands(ndwi1);

# get indexes NDWI2
def getNDWI2(image):
    
    # Normalized difference vegetation index (NDWI2) (Xnir - Xswir2)/(Xnir + Xswir2)
    ndwi2 = image.expression(
      '((NIR - SWIR2) / (NIR + SWIR2 ))', {
      'NIR' : image.select('SR_B5').multiply(0.0001),
      'SWIR2' : image.select('SR_B7').multiply(0.0001),}).rename("ndwi2")
    # Normalized difference vegetation index (NDWI1) (Xnir - Xswir1)/(Xnir + Xswir1)
    

    return image.addBands(ndwi2);


# get indexes NDWI3
def getNDWI3(image):
    
    # Normalized difference vegetation index (NDWI1) (Xgreen - Xnir)/(Xgreen + Xnir)
    ndwi3 = image.expression(
      '((GREEN - NIR) / (GREEN + NIR))', {
      'NIR' : image.select('SR_B5').multiply(0.0001),
      'GREEN' : image.select('SR_B3').multiply(0.0001),}).rename("ndwi3")
    # Normalized difference vegetation index (NDWI1) (Xnir - Xswir1)/(Xnir + Xswir1)
    

    return image.addBands(ndwi3);


def getEVI(image):
    evi = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR' : image.select('SR_B5').multiply(0.0001),
      'RED' : image.select('SR_B4').multiply(0.0001),
      'BLUE': image.select('SR_B2').multiply(0.0001)}).rename('evi');
    return image.addBands(evi);
    

def getSAVI(image):
    savi = image.expression(
     '1.5 * (nir - red) / (nir + red + 0.5)', {
      'nir': image.select('SR_B5').multiply(0.0001),
      'red': image.select('SR_B4').multiply(0.0001)}).rename('savi');
    return image.addBands(savi);


def getK(image):
    
    K1 = image.expression(
      '0.1 * temperature1',{
      'temperature1' : image.select('ST_B10')}).rename('K');
    
    return image.addBands(K1)


def getC(image):
    
    C1 = image.expression(
      '0.1 * temperature1 - 273.15',{
      'temperature1' : image.select('ST_B10')}).rename('C');
    
    return image.addBands(C1)



"""
 * Function to mask clouds based on the pixel_qa band of Landsat 8 SR data.
 * @param {ee.Image} image input Landsat 8 SR image
 * @return {ee.Image} cloudmasked Landsat 8 image
 """


def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)  # this will never fail
    return next_month - timedelta(days=next_month.day)

def monthlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.month == 12:
            next_month = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_month = begin.replace(month=begin.month+1, day=1)
        if next_month > end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
        begin = next_month
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result


def trans_date(input_date):
    t1 = input_date
    t2 = datetime.strptime(t1, '%Y%m%d').strftime('%Y/%m/%d')
    return t2


def cbind(statics):

    all_files = glob.glob(os.path.join(folder_name,"LandsatL2_{}*.csv".format(statics)))

    df_from_each_file = (pd.read_csv(f, sep = ",") for f in all_files)
    df_merged = pd.concat(df_from_each_file, ignore_index = True)
    if 'ndvi' in df_merged.columns.tolist():
        df_merged.rename(columns={'ndvi' : 'ndvi_' + str(statics)}, inplace = True)
    else:
        pass
    if 'evi' in df_merged.columns.tolist():
        df_merged.rename(columns={'evi' : 'evi_' + str(statics)}, inplace = True)
    else:
        pass
    if 'savi' in df_merged.columns.tolist():
        df_merged.rename(columns={'savi' : 'savi_' + str(statics)}, inplace = True)
    else:
        pass
    if 'ndwi1' in df_merged.columns.tolist():
        df_merged.rename(columns={'ndwi1' : 'ndwi1_' + str(statics)}, inplace = True)
    else:
        pass
    if 'ndwi2' in df_merged.columns.tolist():
        df_merged.rename(columns={'ndwi2' : 'ndwi2_' + str(statics)}, inplace = True)
    else:
        pass
    if 'ndwi3' in df_merged.columns.tolist():
        df_merged.rename(columns={'ndwi3' : 'ndwi3_' + str(statics)}, inplace = True)
    else:
        pass
    if 'K' in df_merged.columns.tolist():
        df_merged.rename(columns={'K' : 'K_' + str(statics)}, inplace = True)
    else:
        pass
    
    if 'C' in df_merged.columns.tolist():
        df_merged.rename(columns={'C' : 'C_' + str(statics)}, inplace = True)
    else:
        pass
                
    df_merged = df_merged.groupby(['Date','Doy',file_name.value]).mean().reset_index()


    df_merged.to_csv(folder.selected + '.csv')
    
    shutil.rmtree(folder_name, ignore_errors=True)

""" 
Note that here, the merged file path cannot be the same as the single file path, and must be placed in different folders, 
so this time the following line of code is to output the files to another folder.
"""


' \nNote that here, the merged file path cannot be the same as the single file path, and must be placed in different folders, \nso this time the following line of code is to output the files to another folder.\n'

In [23]:
def zonal(statics):
    time_list =monthlist(star.value, end.value)
    for i in range(0,len(time_list)):
        star_time = time.time()
        
        if os.path.isfile(folder_name +'LandsatL2_{}_{}.csv'.format(statics,time_list[i])) == True:
                
                continue
            
        else:
            clear_output(wait=True)
            Landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                      .filter(ee.Filter.date(time_list[i][0],datetime.strptime(time_list[i][1],"%Y-%m-%d")+ timedelta(days=1))) \
                      .filterBounds(states) \
                      .map(getNDVI) \
                      .map(getEVI) \
                      .map(getSAVI) \
                      .map(getNDWI1) \
                      .map(getNDWI2) \
                      .map(getNDWI3) \
                      .map(getK) \
                      .map(getC) \
                      .select(list(band_name.value)) \
                      .map(lambda image: image.clip(states))


            Landsat = Landsat.toBands()
            out_dir = os.path.expanduser(folder_name)
            out_dem_stats = os.path.join(out_dir, 'LandsatL2_{}_{}.csv'.format(statics,time_list[i]))

            if not os.path.exists(out_dir):
                os.makedirs(out_dir)

            geemap.zonal_statistics(Landsat, states, out_dem_stats, statistics_type=statics, scale=1000)
            data_temp = pd.read_csv(out_dem_stats)
            column_name_list = data_temp.columns.tolist()
            c = []
            d = []
            for k in zip(column_name_list[:]):
                c.append(k[0][12:18])
                d.append(k[0])
            data = []
            for j in range(0, len(column_name_list),len(band_name.value)):            
                if all(m.isdigit() for m in c[j:j+len(band_name.value)]) == True:
                        
                    df = data_temp.loc[:,d[j:j+len(band_name.value)]]
                    df[file_name.value] = data_temp.loc[:,[file_name.value]]
                    text = column_name_list[j][12:20]
                    df.insert(0, 'Date', '')
                    df['Date'] = trans_date(text)
                    df.insert(1, 'Doy', '')
                    df['Doy'] = datetime.strptime(text, '%Y%m%d').strftime('%j')
                    colnames=['Date','Doy']
                    colnames.extend(list(band_name.value))
                    colnames.append(file_name.value)
                    df.columns=[colnames]
                    data.append(df)
                else:
                    continue
                        
            appended_data = pd.concat(data, axis=0,ignore_index = True)
            #cols = appended_data.columns.to_list()
            #cols.insert(len(appended_data.columns), cols.pop(cols.index(file_name)))
            #appended_data = appended_data[cols]
            appended_data.to_csv(out_dem_stats,index=False)#Output the file with date and doy back
            
            end_time = time.time()
            print(end_time - star_time)
            time.sleep(1)

In [26]:
zonal(statics.value)
cbind(statics.value)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Desktop\satellite python code\satellite python code\zonal statistic\data_all\LandsatL2_MEAN_['2020-04-01', '2020-04-22'].csv
61.051372051239014
